# Continuous training with TFX and Google Cloud AI Platform

## Learning Objectives

1.  Use the TFX CLI to build a TFX pipeline.
2.  Deploy a TFX pipeline version without tuning to a hosted AI Platform Pipelines instance.
3.  Create and monitor a TFX pipeline run using the TFX CLI.
4.  Deploy a new TFX pipeline version with tuning enabled to a hosted AI Platform Pipelines instance.
5.  Create and monitor another TFX pipeline run directly in the KFP UI.

In this lab, you use utilize the following tools and services to deploy and run a TFX pipeline on Google Cloud that automates the development and deployment of a TensorFlow 2.3 WideDeep Classifer to predict forest cover from cartographic data:

* The [**TFX CLI**](https://www.tensorflow.org/tfx/guide/cli) utility to build and deploy a TFX pipeline.
* A hosted [**AI Platform Pipeline instance (Kubeflow Pipelines)**](https://www.tensorflow.org/tfx/guide/kubeflow) for TFX pipeline orchestration.
* [**Dataflow**](https://cloud.google.com/dataflow) jobs for scalable, distributed data processing for TFX components.
* A [**AI Platform Training**](https://cloud.google.com/ai-platform/) job for model training and flock management for parallel tuning trials. 
* [**AI Platform Prediction**](https://cloud.google.com/ai-platform/) as a model server destination for blessed pipeline model versions.
* [**CloudTuner**](https://www.tensorflow.org/tfx/guide/tuner#tuning_on_google_cloud_platform_gcp) and [**AI Platform Vizier**](https://cloud.google.com/ai-platform/optimizer/docs/overview) for advanced model hyperparameter tuning using the Vizier algorithm.

You will then create and monitor pipeline runs using the TFX CLI as well as the KFP UI.

### Setup

#### Update lab environment PATH to include TFX CLI and skaffold

In [1]:
import yaml

# Set `PATH` to include the directory containing TFX CLI and skaffold.
PATH=%env PATH
HOME=%env HOME

%env PATH={HOME}/.local/bin:{PATH}

env: PATH=/home/michal/.local/bin:/home/michal/venv/ML-tfx-0.30.0/bin:/home/michal/google-cloud-sdk/bin:/home/michal/anaconda3/condabin:/home/michal/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin


#### Validate lab package version installation

In [2]:
!python -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"
!python -c "import kfp; print('KFP version: {}'.format(kfp.__version__))"

TFX version: 0.30.0
KFP version: 1.4.0


(Optional) If running the above command results in different package versions or you receive an import error, upgrade to the correct versions by running the cell below:

In [ ]:
%pip install --upgrade --user tfx==0.30.0
%pip install --upgrade --user kfp==1.4.0

Note: you may need to restart the kernel to pick up the correct package versions.

## Setup local path to data, train, test folders 

In [3]:
import os
from pathlib import Path

try: notebook_path
except NameError: 
    notebook_path=os.getcwd() # to only run it once

local_data_dirpath = os.path.join(notebook_path, 'data')
local_code_dirpath = os.path.join(notebook_path, 'pipeline')

local_train_dirpath = os.path.join(local_data_dirpath, "train")
local_train_filepath = os.path.join(local_train_dirpath, "train.csv")
local_test_dirpath = os.path.join(local_data_dirpath, "test")
local_test_filepath = os.path.join(local_test_dirpath, "test.csv")


## Load kaggle

In [3]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!rm kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

#### Validate creation of AI Platform Pipelines cluster

Navigate to [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

Note you may have already deployed an AI Pipelines instance during the Setup for the lab series. If so, you can proceed using that instance. If not:

**1.  Create or select an existing Kubernetes cluster (GKE) and deploy AI Platform**. Make sure to select `"Allow access to the following Cloud APIs https://www.googleapis.com/auth/cloud-platform"` to allow for programmatic access to your pipeline by the Kubeflow SDK for the rest of the lab. Also, provide an `App instance name` such as "tfx" or "mlops". 

Validate the deployment of your AI Platform Pipelines instance in the console before proceeding.

## Review: example TFX pipeline design pattern for Google Cloud
The pipeline source code can be found in the `pipeline` folder.

In [4]:
%cd {notebook_path}/pipeline

/home/michal/PycharmProjects/tfx-titanic-training/pipeline


In [5]:
!ls -la

total 180
drwxrwxr-x 8 michal michal  4096 cze 16 17:14 .
drwxrwxr-x 9 michal michal  4096 cze 16 22:51 ..
-rw-rw-r-- 1 michal michal  8339 maj 27 17:06 airflow_runner.py
-rw-rw-r-- 1 michal michal  5883 kwi 29 16:33 beam_dag_runner.py
drwxrwxr-x 2 michal michal  4096 kwi 30 16:38 client
-rw-rw-r-- 1 michal michal  3446 cze 16 00:51 config.py
-rw-rw-r-- 1 michal michal    97 mar 10 12:14 Dockerfile
-rw-rw-r-- 1 michal michal  4310 kwi  9 15:57 features.py
drwxrwxr-x 2 michal michal  4096 kwi 30 16:38 hyperparameters
-rw-rw-r-- 1 michal michal     0 kwi  9 11:32 __init__.py
-rw-rw-r-- 1 michal michal  7075 cze 16 16:46 kubeflow_runner.py
drwxrwxr-x 4 michal michal  4096 maj 29 15:05 lib
-rw-rw-r-- 1 michal michal  5604 maj 29 15:08 local_runner.py
-rw-rw-r-- 1 michal michal 15314 maj 21 19:02 model.py
-rw-rw-r-- 1 michal michal  3452 maj 16 15:19 pipeline_args.py
-rw-rw-r-- 1 michal michal 16049 maj 29 01:03 pipelines.py
-rw-rw-r-- 1 michal michal  4506 kwi 11 02:00 preprocessing.py
drw

The `config.py` module configures the default values for the environment specific settings and the default values for the pipeline runtime parameters. 
The default values can be overwritten at compile time by providing the updated values in a set of environment variables. You will set custom environment variables later on this lab.

The `pipeline.py` module contains the TFX DSL defining the workflow implemented by the pipeline.

The `preprocessing.py` module implements the data preprocessing logic  the `Transform` component.

The `model.py` module implements the training, tuning, and model building logic for the `Trainer` and `Tuner` components.

The `runner.py` module configures and executes `KubeflowDagRunner`. At compile time, the `KubeflowDagRunner.run()` method converts the TFX DSL into the pipeline package in the [argo](https://argoproj.github.io/argo/) format for execution on your hosted AI Platform Pipelines instance.

The `features.py` module contains feature definitions common across `preprocessing.py` and `model.py`.


## Exercise: build your pipeline with the TFX CLI

You will use TFX CLI to compile and deploy the pipeline. As explained in the previous section, the environment specific settings can be provided through a set of environment variables and embedded into the pipeline package at compile time.

### Configure your environment resource settings

Update  the below constants  with the settings reflecting your lab environment. 

- `GCP_REGION` - the compute region for AI Platform Training, Vizier, and Prediction.
- `ARTIFACT_STORE` - An existing GCS bucket. You can use any bucket or use the GCS bucket created during installation of AI Platform Pipelines. The default bucket name will contain the `kubeflowpipelines-` prefix.

In [12]:
# Use the following command to identify the GCS bucket for metadata and pipeline storage.
!gsutil ls

gs://artifacts.cloud-training-281409.appspot.com/
gs://cloud-training-281409/
gs://cloud-training-281409-kubeflowpipelines-default/
gs://kubeflow-storage-goose/


* `CUSTOM_SERVICE_ACCOUNT` - In the gcp console Click on the Navigation Menu. Navigate to `IAM & Admin`, then to `Service Accounts` and use the service account starting with prefix - `'tfx-tuner-caip-service-account'`. This enables CloudTuner and the Google Cloud AI Platform extensions Tuner component to work together and allows for distributed and parallel tuning backed by AI Platform Vizier's hyperparameter search algorithm. Please see the lab setup `README` for setup instructions.

- `ENDPOINT` - set the `ENDPOINT` constant to the endpoint to your AI Platform Pipelines instance. The endpoint to the AI Platform Pipelines instance can be found on the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console. Open the *SETTINGS* for your instance and use the value of the `host` variable in the *Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD* section of the *SETTINGS* window. The format is `'...pipelines.googleusercontent.com'`.

### GCP run environment settings

In [30]:
#TODO: Set your environment resource settings here for GCP_REGION, ARTIFACT_STORE_URI, ENDPOINT, and CUSTOM_SERVICE_ACCOUNT.
GCP_REGION = 'us-central1'
ARTIFACT_STORE_URI = 'gs://cloud-training-281409-kubeflowpipelines-default'
ENDPOINT = 'https://c281ee5ff79be53-dot-us-central1.pipelines.googleusercontent.com'
CUSTOM_SERVICE_ACCOUNT = 'tfx-tuner-service-account@cloud-training-281409.iam.gserviceaccount.com'

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

USE_GS=True
USE_AI_PLATFORM=True


In [31]:
# Set your resource settings as environment variables. These override the default values in pipeline/config.py.
%env GCP_REGION={GCP_REGION}
%env ARTIFACT_STORE_URI={ARTIFACT_STORE_URI}
%env CUSTOM_SERVICE_ACCOUNT={CUSTOM_SERVICE_ACCOUNT}
%env PROJECT_ID={PROJECT_ID}

%env USE_GS={USE_GS}
%env USE_AI_PLATFORM={USE_AI_PLATFORM}

env: GCP_REGION=us-central1
env: ARTIFACT_STORE_URI=gs://cloud-training-281409-kubeflowpipelines-default
env: CUSTOM_SERVICE_ACCOUNT=tfx-tuner-service-account@cloud-training-281409.iam.gserviceaccount.com
env: PROJECT_ID=cloud-training-281409
env: USE_GS=True
env: USE_AI_PLATFORM=True


### Kubeflow run on local cluster

In [43]:
KUBEFLOW_ARTIFACT_STORE = os.path.join(os.sep, HOME, 'kubeflow-artifact-store')
#ARTIFACT_STORE_URI = KUBEFLOW_ARTIFACT_STORE


GCP_REGION = 'us-central1'
ARTIFACT_STORE_URI = 'gs://cloud-training-281409-kubeflowpipelines-default'
ENDPOINT = 'http://localhost:8080'
CUSTOM_SERVICE_ACCOUNT = 'tfx-tuner-service-account@cloud-training-281409.iam.gserviceaccount.com'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

USE_GS=True
USE_AI_PLATFORM=False


In [44]:
# Set your resource settings as environment variables. These override the default values in pipeline/config.py.
%env GCP_REGION={GCP_REGION}
%env ARTIFACT_STORE_URI={ARTIFACT_STORE_URI}
%env CUSTOM_SERVICE_ACCOUNT={CUSTOM_SERVICE_ACCOUNT}
%env PROJECT_ID={PROJECT_ID}

%env USE_GS={USE_GS}
%env USE_AI_PLATFORM={USE_AI_PLATFORM}

env: GCP_REGION=us-central1
env: ARTIFACT_STORE_URI=gs://cloud-training-281409-kubeflowpipelines-default
env: CUSTOM_SERVICE_ACCOUNT=tfx-tuner-service-account@cloud-training-281409.iam.gserviceaccount.com
env: PROJECT_ID=cloud-training-281409
env: USE_GS=True
env: USE_AI_PLATFORM=False


### Set the compile time settings to first create a pipeline version without hyperparameter tuning

Default pipeline runtime environment values are configured in the pipeline folder `config.py`. You will set their values directly below:

* `PIPELINE_NAME` - the pipeline's globally unique name. For each pipeline update, each pipeline version uploaded to KFP will be reflected on the `Pipelines` tab in the `Pipeline name > Version name` dropdown in the format `PIPELINE_NAME_datetime.now()`.

* `MODEL_NAME` - the pipeline's unique model output name for AI Platform Prediction. For multiple pipeline runs, each pushed blessed model will create a new version with the format `'v{}'.format(int(time.time()))`.

* `DATA_ROOT_URI` - the URI for the raw lab dataset `gs://workshop-datasets/covertype/small`.

* `CUSTOM_TFX_IMAGE` - the image name of your pipeline container build by skaffold and published by `Cloud Build` to `Cloud Container Registry` in the format `'gcr.io/{}/{}'.format(PROJECT_ID, PIPELINE_NAME)`.

* `RUNTIME_VERSION` - the TensorFlow runtime version. This lab was built and tested using TensorFlow `2.3`.

* `PYTHON_VERSION` - the Python runtime version. This lab was built and tested using Python `3.7`.

* `USE_KFP_SA` - The pipeline can run using a security context of the GKE default node pool's service account or the service account defined in the `user-gcp-sa` secret of the Kubernetes namespace hosting Kubeflow Pipelines. If you want to use the `user-gcp-sa` service account you change the value of `USE_KFP_SA` to `True`. Note that the default AI Platform Pipelines configuration does not define the `user-gcp-sa` secret.

* `ENABLE_TUNING` - boolean value indicating whether to add the `Tuner` component to the pipeline or use hyperparameter defaults. See the `model.py` and `pipeline.py` files for details on how this changes the pipeline topology across pipeline versions. You will create pipeline versions without and with tuning enabled in the subsequent lab exercises for comparison.

In [45]:
CODE_FOLDER = local_code_dirpath
PIPELINE_NAME = 'tfx-titanic-training'
MODEL_NAME = 'tfx_titanic-classifier'
#DATA_ROOT_URI = local_train_dirpath
DATA_ROOT_URI = 'gs://cloud-training-281409-kubeflowpipelines-default/tfx-template/data/titanic'

RUNTIME_VERSION = '2.4'
PYTHON_VERSION = '3.8'
USE_KFP_SA=True
ENABLE_CACHE=False

EPOCHS = '50'
TRAIN_STEPS = '2000'
EVAL_STEPS = '100'
TRAIN_BATCH_SIZE = '64'
EVAL_BATCH_SIZE = '64'

ENABLE_TUNING=True
TUNER_STEPS = '400'
MAX_TRIALS = '50'

KUBEFLOW_TFX_IMAGE = 'tensorflow/tfx:0.30.0'

CUSTOM_TFX_IMAGE = 'gcr.io/{}/{}'.format('local_kubeflow', PIPELINE_NAME)

LOCAL_RUN = 'False'



In [46]:
%env CODE_FOLDER={CODE_FOLDER}
%env PIPELINE_NAME={PIPELINE_NAME}
%env MODEL_NAME={MODEL_NAME}
%env DATA_ROOT_URI={DATA_ROOT_URI}

%env RUNTIME_VERSION={RUNTIME_VERSION}
%env PYTHON_VERSION={PYTHON_VERSION}
%env USE_KFP_SA={USE_KFP_SA}
%env ENABLE_CACHE={ENABLE_CACHE}

%env TRAIN_STEPS={TRAIN_STEPS}
%env EVAL_STEPS={EVAL_STEPS}
%env TRAIN_BATCH_SIZE={TRAIN_BATCH_SIZE}
%env EVAL_BATCH_SIZE={EVAL_BATCH_SIZE}


%env ENABLE_TUNING={ENABLE_TUNING}
%env TUNER_STEPS={TUNER_STEPS}
%env MAX_TRIALS={MAX_TRIALS}

%env KUBEFLOW_TFX_IMAGE={KUBEFLOW_TFX_IMAGE}
%env CUSTOM_TFX_IMAGE={CUSTOM_TFX_IMAGE}

%env LOCAL_RUN={LOCAL_RUN}



env: CODE_FOLDER=/home/michal/PycharmProjects/tfx-titanic-training/pipeline
env: PIPELINE_NAME=tfx-titanic-training
env: MODEL_NAME=tfx_titanic-classifier
env: DATA_ROOT_URI=gs://cloud-training-281409-kubeflowpipelines-default/tfx-template/data/titanic
env: RUNTIME_VERSION=2.4
env: PYTHON_VERSION=3.8
env: USE_KFP_SA=True
env: ENABLE_CACHE=False
env: TRAIN_STEPS=2000
env: EVAL_STEPS=100
env: TRAIN_BATCH_SIZE=64
env: EVAL_BATCH_SIZE=64
env: ENABLE_TUNING=True
env: TUNER_STEPS=400
env: MAX_TRIALS=50
env: KUBEFLOW_TFX_IMAGE=tensorflow/tfx:0.30.0
env: CUSTOM_TFX_IMAGE=gcr.io/local_kubeflow/tfx-titanic-training
env: LOCAL_RUN=False


## Copy data

In [17]:
import os
from pathlib import Path

_data_root = os.path.join(".", 'data')
_train_dirpath = os.path.join(_data_root, "train")
_train_filepath = os.path.join(_train_dirpath, "train.csv")
_test_dirpath = os.path.join(_data_root, "test")
_test_filepath = os.path.join(_test_dirpath, "test.csv")

## Download data from kaggle, unzip it and copy it to data folder

In [20]:
!kaggle competitions download -c titanic -p {_data_root} --force
!unzip -o {_data_root}/"titanic.zip" -d {_data_root}
!cp {_data_root}/"train.csv" {_train_filepath}
!cp {_data_root}/"test.csv" {_test_filepath}

# clean up
!rm  {_data_root}/*.csv  {_data_root}/*.zip

  0%|                                               | 0.00/34.1k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 6.56MB/s]
Archive:  ./data/titanic.zip
  inflating: ./data/gender_submission.csv  
  inflating: ./data/test.csv         
  inflating: ./data/train.csv        


Let's upload our sample data to GCS bucket so that we can use it in our pipeline later.

Copy csv data to bucket

In [18]:
!gsutil cp {notebook_path}/data/train/train.csv gs://cloud-training-281409-kubeflowpipelines-default/tfx-template/data/titanic/data.csv




Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Copying file:///home/michal/PycharmProjects/tfx-titanic-training/data/train/train.csv [Content-Type=text/csv]...
- [1 files][ 58.9 KiB/ 58.9 KiB]                                                
Operation completed over 1 objects/58.9 KiB.                                     


## Change directory to pipeline

In [23]:
%cd {notebook_path}/pipeline

/home/michal/PycharmProjects/tfx-titanic-training/pipeline


### Compile your pipeline code

You can build and upload the pipeline to the AI Platform Pipelines instance in one step, using the `tfx pipeline create` command. The `tfx pipeline create` goes through the following steps:
- (Optional) Builds the custom image to that provides a runtime environment for TFX components or uses the latest image of the installed TFX version 
- Compiles the pipeline code into a pipeline package 
- Uploads the pipeline package via the `ENDPOINT` to the hosted AI Platform instance.

As you debug the pipeline DSL, you may prefer to first use the `tfx pipeline compile` command, which only executes the compilation step. After the DSL compiles successfully you can use `tfx pipeline create` to go through all steps.

In [47]:
!tfx pipeline compile --engine kubeflow --pipeline_path kubeflow_runner.py

2021-06-22 16:31:19.943129: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2021-06-22 16:31:19.943292: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Compiling pipeline
INFO:absl:PIPELINE_ROOT=gs://cloud-training-281409-kubeflowpipelines-default/tfx-titanic-training/{{workflow.uid}}
INFO:absl:pipeline_name: tfx-titanic-training
INFO:absl:pipeline root: gs://cloud-training-281409-kubeflowpipelines-default/tfx-titanic-training/{{workflow.uid}}
INFO:absl:data_root_uri for training: gs://cloud-training-281409-kubeflowpipelines-default/tfx-template/data/titanic
INFO:absl:train_steps for training: 2000
INFO:absl:tuner_steps for tuning: 400
INFO:absl:eval_steps for evaluating: 100
INFO:absl:os d

Note: you should see a `{PIPELINE_NAME}.tar.gz` file appear in your current pipeline directory.

In [48]:
import kfp

#ENDPOINT = 'http://localhost:8080'
#ENDPOINT = 'http://localhost:8080'

client = kfp.Client(host=ENDPOINT)

client.list_pipelines()

#client.KUBE_PROXY_PATH

{'next_page_token': None,
 'pipelines': [{'created_at': datetime.datetime(2021, 6, 16, 20, 36, 12, tzinfo=tzutc()),
                'default_version': {'code_source_url': None,
                                    'created_at': datetime.datetime(2021, 6, 16, 20, 36, 12, tzinfo=tzutc()),
                                    'id': '66c58f50-0330-4719-bd88-3aa472eb7994',
                                    'name': '[Demo] XGBoost - Iterative model '
                                            'training',
                                    'package_url': None,
                                    'parameters': None,
                                    'resource_references': [{'key': {'id': '66c58f50-0330-4719-bd88-3aa472eb7994',
                                                                     'type': 'PIPELINE'},
                                                             'name': None,
                                                             'relationship': 'OWNER'}]},
             

## Exercise: deploy your pipeline container to AI Platform Pipelines with TFX CLI

After the pipeline code compiles without any errors you can use the `tfx pipeline create` command to perform the full build and deploy the pipeline. You will deploy your compiled pipeline container hosted on Google Container Registry e.g. `gcr.io/[PROJECT_ID]/tfx_covertype_continuous_training` to run on AI Platform Pipelines with the TFX CLI.

In [14]:
#!tfx pipeline create --pipeline_path=kubeflow_runner.py --endpoint={ENDPOINT} \
#--build_target_image={CUSTOM_TFX_IMAGE}
!tfx pipeline create --pipeline_path=kubeflow_runner.py --engine kubeflow --endpoint={ENDPOINT}
#--build_target_image={CUSTOM_TFX_IMAGE}

2021-06-16 22:54:45.795373: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2021-06-16 22:54:45.795424: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Creating pipeline
INFO:absl:PIPELINE_ROOT=/home/michal/kubeflow-artifact-store/tfx-titanic-training/{{workflow.uid}}
INFO:absl:pipeline_name: tfx-titanic-training
INFO:absl:pipeline root: /home/michal/kubeflow-artifact-store/tfx-titanic-training/{{workflow.uid}}
INFO:absl:data_root_uri for training: /home/michal/PycharmProjects/tfx-titanic-training/data/train
INFO:absl:train_steps for training: 2000
INFO:absl:tuner_steps for tuning: 400
INFO:absl:eval_steps for evaluating: 100
INFO:absl:os default list dir: ['__init__.py', 'schema', 'local_r

writing manifest file 'tfx_user_code_Tuner.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Tuner.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Tuner.egg-info/SOURCES.txt'
Copying tfx_user_code_Tuner.egg-info to /tmp/tmpseuch2g3/tfx_user_code_Tuner-0.0+0a6a320852b6ce12f52a47d741a665a38c58498e157bb473a99d923bc4e7f157.egg-info
running install_scripts
creating /tmp/tmpseuch2g3/tfx_user_code_Tuner-0.0+0a6a320852b6ce12f52a47d741a665a38c58498e157bb473a99d923bc4e7f157.dist-info/WHEEL
creating '/tmp/tmp1hggifi0/tfx_user_code_Tuner-0.0+0a6a320852b6ce12f52a47d741a665a38c58498e157bb473a99d923bc4e7f157-py3-none-any.whl' and adding '/tmp/tmpseuch2g3' to it
adding 'airflow_runner.py'
adding 'beam_dag_runner.py'
adding 'config.py'
adding 'features.py'
adding 'kubeflow_runner.py'
adding 'local_runner.py'
adding 'model.py'
adding 'pipeline_args.py'
adding 'pipelines.py'
adding 'preprocessing.py'
adding 'tfx_user_code_Tuner-0.0+0a6a320852b6ce12f52a47d741a665a38c58498e157bb473a99d9

                value: INT
              }
              properties {
                key: "split_names"
                value: STRING
              }
            }
          }
        }
      }
    }
    parameters {
      parameters {
        key: "exclude_splits"
        value {
          field_value {
            string_value: "[]"
          }
        }
      }
    }
    upstream_nodes: "StatisticsGen"
    upstream_nodes: "import_user_schema"
    execution_options {
      caching_options {
      }
    }
  }
}
nodes {
  pipeline_node {
    node_info {
      type {
        name: "tfx.components.schema_gen.component.SchemaGen"
      }
      id: "SchemaGen"
    }
    contexts {
      contexts {
        type {
          name: "pipeline"
        }
        name {
          field_value {
            string_value: "tfx-titanic-training"
          }
        }
      }
      contexts {
        type {
          name: "pipeline_run"
        }
        name {
          runtime_parameter {
        

INFO:absl:Adding upstream dependencies for component schemagen
INFO:absl:   ->  Component: statisticsgen
INFO:absl:Adding upstream dependencies for component tuner
INFO:absl:   ->  Component: transform
INFO:absl:Adding upstream dependencies for component trainer
INFO:absl:   ->  Component: tuner
INFO:absl:   ->  Component: import-user-schema
INFO:absl:   ->  Component: transform
INFO:absl:Adding upstream dependencies for component evaluator
INFO:absl:   ->  Component: latest-blessed-model-resolver
INFO:absl:   ->  Component: csvexamplegen
INFO:absl:   ->  Component: trainer
INFO:absl:Adding upstream dependencies for component infravalidator
INFO:absl:   ->  Component: csvexamplegen
INFO:absl:   ->  Component: trainer
INFO:absl:Adding upstream dependencies for component pusher
INFO:absl:   ->  Component: infravalidator
INFO:absl:   ->  Component: trainer
INFO:absl:   ->  Component: evaluator
Please access the pipeline detail page at http://localhost:8080/#/pipelines/details/8670e379-208

**Hint**: review the [TFX CLI documentation](https://www.tensorflow.org/tfx/guide/cli#create) on the "pipeline group" to create your pipeline. You will need to specify the `--pipeline_path` to point at the pipeline DSL and runner defined locally in `runner.py`, `--endpoint`, and `--build_target_image` arguments using the environment variables specified above.

Note: you should see a `build.yaml` file in your pipeline folder created by skaffold. The TFX CLI compile triggers a custom container to be built with skaffold using the instructions in the `Dockerfile`.

If you need to redeploy the pipeline you can first delete the previous version using `tfx pipeline delete` or you can update the pipeline in-place using `tfx pipeline update`.

To delete the pipeline:

`tfx pipeline delete --pipeline_name {PIPELINE_NAME} --endpoint {ENDPOINT}`

To update the pipeline:

`tfx pipeline update --pipeline_path runner.py --endpoint {ENDPOINT}`

In [ ]:
!tfx pipeline delete --pipeline_name {PIPELINE_NAME} --endpoint {ENDPOINT}

In [49]:
!tfx pipeline update --pipeline_path kubeflow_runner.py --engine kubeflow --endpoint {ENDPOINT}

2021-06-22 16:32:07.717953: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2021-06-22 16:32:07.718003: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Updating pipeline
INFO:absl:PIPELINE_ROOT=gs://cloud-training-281409-kubeflowpipelines-default/tfx-titanic-training/{{workflow.uid}}
INFO:absl:pipeline_name: tfx-titanic-training
INFO:absl:pipeline root: gs://cloud-training-281409-kubeflowpipelines-default/tfx-titanic-training/{{workflow.uid}}
INFO:absl:data_root_uri for training: gs://cloud-training-281409-kubeflowpipelines-default/tfx-template/data/titanic
INFO:absl:train_steps for training: 2000
INFO:absl:tuner_steps for tuning: 400
INFO:absl:eval_steps for evaluating: 100
INFO:absl:os de

### Create and monitor a pipeline run with the TFX CLI

After the pipeline has been deployed, you can trigger and monitor pipeline runs using TFX CLI.

*Hint*: review the [TFX CLI documentation](https://www.tensorflow.org/tfx/guide/cli#run_group) on the "run group".

In [39]:
!tfx run create --pipeline_name={PIPELINE_NAME} --engine kubeflow --endpoint={ENDPOINT}

2021-06-18 13:24:30.832593: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2021-06-18 13:24:30.832651: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Creating a run for pipeline: tfx-titanic-training
Run created for pipeline: tfx-titanic-training
+======================+======================================+========+===========================+===========================================================================+
| pipeline_name        | run_id                               | status | created_at                | link                                                                      |
+======================+======================================+========+========================

To view the status of existing pipeline runs:

In [40]:
!tfx run list --pipeline_name {PIPELINE_NAME} --engine kubeflow --endpoint {ENDPOINT}

2021-06-18 13:26:47.295516: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2021-06-18 13:26:47.295579: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
CLI
Listing all runs of pipeline: tfx-titanic-training
+======================+======================================+========+===========================+===========================================================================+
| pipeline_name        | run_id                               | status | created_at                | link                                                                      |
+======================+======================================+========+===========================+==========================================

To retrieve the status of a given run:

In [ ]:
RUN_ID='[YOUR RUN ID]'

!tfx run status --pipeline_name {PIPELINE_NAME} --run_id {RUN_ID} --endpoint {ENDPOINT}

### Compile your pipeline code

In [19]:
!tfx pipeline compile --engine kubeflow --pipeline_path runner.py

CLI
Compiling pipeline
Pipeline compiled successfully.
Pipeline package path: /home/jupyter/ml-gcp-pipeline/tfx_titanic_pipeline/pipeline/tfx-titanic-training.tar.gz


### Deploy your pipeline container to AI Platform Pipelines with the TFX CLI

In [ ]:
!tfx pipeline update --pipeline_path kubeflow_runner.py --endpoint {ENDPOINT}

### Trigger a pipeline run from the Kubeflow Pipelines UI

On the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page, click `OPEN PIPELINES DASHBOARD`. A new browser tab will open. Select the `Pipelines` tab to the left where you see the `PIPELINE_NAME` pipeline you deployed previously. You should see 2 pipeline versions. 

Click on the most recent pipeline version with tuning enabled which will open up a window with a graphical display of your TFX pipeline directed graph. 

Next, click the `Create a run` button. Verify the `Pipeline name` and `Pipeline version` are pre-populated and optionally provide a `Run name` and `Experiment` to logically group the run metadata under before hitting `Start` to trigger the pipeline run.

## Important

A full pipeline run with tuning enabled will take about 50 minutes and can be executed in parallel while the previous pipeline run without tuning continues running. 

Take the time to review the pipeline metadata artifacts created in the GCS artifact repository for each component including data splits, your Tensorflow SavedModel, model evaluation results, etc. as the pipeline executes. In the GCP console, you can also view the Dataflow jobs for pipeline data processing as well as the AI Platform Training jobs for model training and tuning.

When your pipelines runs are complete, review your model versions on Cloud AI Platform Prediction and model evaluation metrics. Did your model performance improve with hyperparameter tuning?

## Next Steps

In this lab, you learned how to build and deploy a TFX pipeline with the TFX CLI and then update, build and deploy a new pipeline with automatic hyperparameter tuning. You practiced triggered continuous pipeline runs using the TFX CLI as well as the Kubeflow Pipelines UI.


In the next lab, you will construct a Cloud Build CI/CD workflow that further automates the building and deployment of the TensorFlow WideDeep Classifer pipeline code introduced in this lab.

## License

<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>